# Projeto 1 - Ciência dos Dados

Nome: Vitor Fortes Giuliano Riccetti 

Nome: Rodrigo Paoliello de Medeiros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [114]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [115]:
import os

filename = 'Chuteira_Puma.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Chuteira_Puma.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [116]:
# Faz a leitura do excel 
train = pd.read_excel(filename)
train.head(5)

train["Relevante"]= train["Relevante"].astype(str)
train["Irrelevante"]= train["Irrelevante"].astype(str)



In [117]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevante,Irrelevante
0,NaN,@gathlos de níver atrasado quero uma chuteira ...,"""o rei está de volta!""\n\na puma confirmou hoj..."
1,NaN,@neymarjrdepre @neymarjr @puma essa chuteira é...,"@bergonsi_carlos chuteira preta da puma, é ami..."
2,NaN,era só essa chuteira da puma na minha vida,"@bernardoqac chuteira preta da puma e barões, ..."
3,NaN,@manassescruz1 @juancrf19 @futebol_info @puma_...,@danilindo____ @neymarjrdepre @breninhoo___ @n...
4,NaN,só a chuteira da puma na minha vida mesmo.,@futtmais posta as fotos d ney com a nova chut...


In [118]:
# dividimos em duas categorias relevante e irrelevante:

Relevantes = ''.join(train.Relevante)

Irrelevantes = ' '.join(train.Irrelevante)


In [119]:
# Função para limpar a string
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    text = re.sub(r'@\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    #text = re.sub(r'kk\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;“”_]' # Note que os sinais [] são delimitadores de um conjunto.

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed


In [120]:
# Retira as pontuações e os links

Lista_R_clean = cleanup(Relevantes.lower())
#Lista_R_clean



In [121]:
# Retira as pontuações e os links

Lista_IR_clean = cleanup(Irrelevantes.lower())
#Lista_IR_clean


In [122]:
# Serie de palavras Relevantes
Palavras_relevantes = pd.Series(Lista_R_clean.split())

len(Palavras_relevantes)

1745

In [123]:
# Serie de palavras Irrelevantes
Palavras_irrelevantes = pd.Series(Lista_IR_clean.split())

len(Palavras_irrelevantes)

2089

In [124]:
# Tabela absoluta dos Relevantes.
tabela_absoluta_relevantes = Palavras_relevantes.value_counts(ascending=False)


tabela_absoluta_relevantes.head(30)

chuteira    136
puma        122
da           99
a            49
que          42
é            38
uma          32
do           26
de           25
e            25
o            23
eu           23
essa         23
preta        22
não          19
neymar       19
comprar      16
com          16
mas          15
vou          15
pra          14
feia         13
ney          12
um           12
vai          11
só           11
q            11
mais         10
tem          10
no           10
dtype: int64

In [125]:
# Tabela absoluta dos Irrelevantes.
tabela_absoluta_irrelevantes = Palavras_irrelevantes.value_counts(ascending=False)


tabela_absoluta_irrelevantes.head(30)

chuteira    154
da          125
puma        113
a            83
neymar       56
o            55
e            42
preta        42
com          42
de           38
do           33
uma          28
que          26
é            21
não          21
nike         20
pra          19
ney          17
hexa         17
vai          17
ele          16
na           14
q            13
tá           13
essa         13
cara         13
só           11
agora        11
um           11
eu           11
dtype: int64

In [126]:
# Frequencia relativa dos Relevantes 

tabela_relativa_relevantes = Palavras_relevantes.value_counts(True)

tabela_relativa_relevantes

chuteira       0.077937
puma           0.069914
da             0.056734
a              0.028080
que            0.024069
                 ...   
kkkkkqueria    0.000573
geral          0.000573
renovar        0.000573
animal         0.000573
pagar          0.000573
Length: 572, dtype: float64

In [127]:
tabela_relativa_relevantes.sum()

1.0

In [128]:
# Frequencia relativa dos Irrelevantes

tabela_relativa_irrelevantes = Palavras_irrelevantes.value_counts(True)

tabela_relativa_relevantes

chuteira       0.077937
puma           0.069914
da             0.056734
a              0.028080
que            0.024069
                 ...   
kkkkkqueria    0.000573
geral          0.000573
renovar        0.000573
animal         0.000573
pagar          0.000573
Length: 572, dtype: float64

In [ ]:
tabela_relativa_irrelevantes.sum()

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [101]:
P(palavra|Relevante) = 

P(palavra) = 

P(Relevantes) = 

SyntaxError: invalid syntax (<ipython-input-101-953eac6f7a48>, line 1)

In [ ]:
P(Relevante|palavra) = (P(palavra|Relevante) * P(Relevantes)) / P(palavra)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**